In [7]:
from tqdm import tqdm_notebook, tnrange
import numpy as np
from random import shuffle
import pandas as pd
import os
import random

# Additional function definitions

In [77]:
def reconstruct_images(plot_id):
    '''
    Takes a plot ID and subsets the input pd.DataFrame to that plot ID
    returns a (14, 14) array-like list with binary labels
    # LAT, LONG, TREE
    # 42.11, 1.02, 0
    # 42.1102, 1.02, 1
    
    '''
    subs = df[df['PLOT_ID'] == plot_id]
    rows = []
    lats = reversed(sorted(subs['LAT'].unique()))
    for i, val in enumerate(lats):
        subs_lat = subs[subs['LAT'] == val]
        subs_lat = subs_lat.sort_values('LON', axis = 0)
        rows.append(list(subs_lat['TREE']))
    return rows

# Data loading

In [9]:
source = 'test'
sentinel_1 = True
s2_path = "../{}-super/".format(source)
s1_path = "../{}-s1/".format(source)
csv_path = "../{}-csv/".format(source)
output_path = "../{}-processed/".format(source)

### To me this seemed like not a check for whether there is labelled y data for it, but whether there is data from sentinel 2 for it? or what is the difference between s2 and s1? what is s2? And does s2 only contain labelled rows?

cause else he could have done this to get rid of non-labelled rows:



In [9]:
dani = pd.read_csv("../test-csv/cameroon-test.csv").drop('IMAGERY_TITLE', axis = 1)
dani[dani['TREE'].notna()].head(20)



### what is s2 - test-super? It has shape (24,16,16,15), so is it 15 bands for each pixel? But then what is the 24?
### And also below in the df JB creates there are only 14 rows per LAT, i.e. 14 pixels...?
### What is the csv for?

In [18]:
s2 = np.load("../test-super/135505808.npy")
s2.shape
s2[0][0][0]

array([0.1084197 , 0.15956987, 0.22117239, 0.27713728, 0.24362932,
       0.2551793 , 0.26495138, 0.275739  , 0.30990466, 0.27289602,
       0.06345104, 0.07818227, 0.34759885, 0.07568269, 0.83566904],
      dtype=float32)

In [11]:
# For either train or test data, loop through each plot and determine whether there is
# labelled Y data for it -- returning one dataframe for the entire data set

dfs = []
for i in os.listdir(csv_path):
    if ".csv" in i:
        print(i)
        df = pd.read_csv(csv_path + i).drop('IMAGERY_TITLE', axis = 1)
        df['country'] = i.split(".")[0]
        dfs.append(df)

for i in range(len(dfs)):
    if "PL_PLOTID" in dfs[i].columns:
            dfs[i] = dfs[i].drop("PL_PLOTID", axis = 1)
    if 'STACKINGPROFILEDG' in dfs[i].columns:
        dfs[i] = dfs[i].drop('STACKINGPROFILEDG', axis = 1)
    if 'IMAGERYYEARDG' in dfs[i].columns:
        dfs[i] = dfs[i].drop('IMAGERYYEARDG', axis = 1)

df = pd.concat(dfs, ignore_index = True)
df = df.dropna(axis = 0)

existing = [int(x[:-4]) for x in os.listdir(s2_path) if ".DS" not in x]
df = df[df['PLOT_ID'].isin(existing)]
plot_ids = sorted(df['PLOT_ID'].unique())

india-test.csv
lac-test.csv
lac-north-test.csv
cameroon-test.csv
africa-east-test.csv
ghana-test.csv
global-test.csv
kenya-test.csv
lac-south-test.csv
ethiopia-test.csv
ghana-test-large.csv
africaeast-test.csv
africawest-test.csv


In [15]:
df.head(30)

,PLOT_ID,SAMPLE_ID,LON,LAT,FLAGGED,ANALYSES,USER_ID,COLLECTION_TIME,ANALYSIS_DURATION,TREE,country
0,135804022,494271806,74.880908,19.271123,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,1.0,india-test
1,135804022,494271807,74.880908,19.271208,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,1.0,india-test
2,135804022,494271808,74.880908,19.271293,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,1.0,india-test
3,135804022,494271809,74.880908,19.271377,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,0.0,india-test
4,135804022,494271810,74.880908,19.271462,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,0.0,india-test
5,135804022,494271811,74.880908,19.271547,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,0.0,india-test
6,135804022,494271812,74.880908,19.271632,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,0.0,india-test
7,135804022,494271813,74.880908,19.271717,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,0.0,india-test
8,135804022,494271814,74.880908,19.271801,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,0.0,india-test
9,135804022,494271815,74.880908,19.271886,False,1,jo.m.brandt@gmail.com,2020-02-04 16:33:08.763258,171.3,0.0,india-test


In [20]:
s1d = np.load("../test-s1/135505808.npy")
s1d.shape
#s1d[0][0][0]

(24, 16, 16, 2)

In [107]:
# Initiate empty lists to store the X and Y data in
data_x, data_y, lengths = [], [], []
    
countries = {}
count = 0
to_remove = []
# Iterate over each plot
for i in tnrange(len(plot_ids)):
    skip = True if sentinel_1 else False
    # Load the sentinel imagery
    if (str(plot_ids[i]) + ".npy")in os.listdir(s2_path): # ??? Isn't this already guaranteed from the above block where he looks for 'existing'?
        country = df[df['PLOT_ID'] == plot_ids[i]]['country'].unique()
        if str(country[0]) not in countries.keys():
            countries[str(country[0])] = [count, count]
        countries[str(country[0])][1] = count
        x = np.load(s2_path + str(plot_ids[i]) + ".npy")
        if sentinel_1 and os.path.isfile(s1_path + str(plot_ids[i]) + ".npy"):
            skip = False
            s1 = np.load(s1_path + str(plot_ids[i]) + ".npy")
            x = np.concatenate([x, s1], axis = -1)
        count += 1
    y = reconstruct_images(plot_ids[i])
    if not skip:
        lengths.append(x.shape[0])
        data_x.append(x)
        data_y.append(y)
print("Finished data loading")

data_x = np.stack(data_x)
data_y = np.stack(np.array(data_y))
lengths = np.stack(lengths)


Finished data loading


In [111]:
len(data_y)
data_y.shape

(684, 14)

In [105]:
len(data_y)

def flatten(L):
    '''(list) -> list
    Returns a flattened version of nested list L
    >>> flatten([1,[2,[3,4]],5])
    [1, 2, 3, 4, 5]
    '''
    # base case: list with one element
    if len(L) == 1:
        if type(L[0]) == list:
            result = flatten(L[0])
        else:
            result = L
    elif type(L[0]) == list:
        result = flatten(L[0]) + flatten(L[1:])
    else:
        result = [L[0]] + flatten(L[1:])
    return result

len(flatten(data_y))

148960

In [110]:
data_x = np.stack(data_x)
data_y2 = np.reshape(flatten(data_y), (760,14, 14))

data_y2.shape

ValueError: cannot reshape array of size 14 into shape (760,14,14)

In [30]:
[len(x) for x in data_y]

counter = 0
for y in data_y:
    if len(y) == 14:
        counter += 1
    else:
        print(counter)

In [53]:
data_x.shape
type(data_x)


numpy.ndarray

In [83]:
# data y shape: 684, 14, 14
# why is y 14 * 14 and x 16*16?
data_y

[[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0],
  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [74]:
y = reconstruct_images(135804022)
y

[array([0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])]

# Data writing

In [17]:
# This writes a (N_samples, time, width, height, channels) X data that is not standardized
# Training 4500, 24, 16, 16, 16
# and a (N_samples, width, height) Y data where the Y is a 0 / 1 binary tree presence
# The X data is 16x16, and the Y data is 14x14 -- so the X data incorporates an extra boundary
# pixel, which should be downsampled before output layer with a convolution layer with no padding

# The X data channels are as below:
# [B02,B03,B04,B05,B06,B07, B08, B8A,B11,B12, SLOPE, EVI, MSAVI2, Bare soil index,
#    Sentinel 1 VV, Sentinel 1 VH]

# GAN 

# Pretraining

# Normalization
#     layer norm,
#     batch renorm
#     batch norm, batch renormalization, instance normalization,
#     self normalization - https://arxiv.org/abs/1706.02515

# Loss functions
#   - binary cross entropy
#   - boundary loss
#   - lovasz softmax
#   - directly optimize dice score
#   - hausdorff distance metrics
#   - focal loss

# Baseline ResNet, InceptionNet, XceptionNet


if source == 'train':
    np.save(output_path + "data_x_l2a_processed.npy", data_x)
    np.save(output_path + "data_y_l2a_processed.npy", np.array(data_y))
    np.save(output_path + "length_l2a_processed.npy", np.array(lengths))
if source == 'test' or source == "project":
    print("Writing test data")
    np.save(output_path + "test_x_l2a_processed.npy", data_x)
    np.save(output_path + "test_y_l2a_processed.npy", data_y)
    np.save(output_path + "test_length_l2a_processed.npy", lengths)